Import Dependencies

In [ ]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
import gc
import torch

Empty GPU Cache

In [ ]:
gc.collect()
torch.cuda.empty_cache()

Avoid OOM errors by setting GPU Memory Consumption Growth

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

Define Paths to File

In [ ]:
ambulance = os.path.join('data', 'ambulance')
traffic = os.path.join('data', 'traffic')

Build Dataloading Function

In [ ]:
def load_wav_16k_mono(filename):
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=48000)
    return wav

Define paths to Positive and Negative Data

In [ ]:
POS = os.path.join('data','ambulance')
NEG = os.path.join('data','traffic')

Create Tensorflow Data

In [ ]:
pos = tf.data.Dataset.list_files(POS+'\*.wav')
neg = tf.data.Dataset.list_files(NEG+'\*.wav')

Add Labels and Combine Positive and Negative Samples

In [ ]:
positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))
data = positives.concatenate(negatives)

Building Preprocessing Function

In [ ]:
def preprocess(file_path, label): 
    wav = load_wav_16k_mono(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

Test out the function and viz the Spectrogram

In [ ]:
filepath, label = positives.shuffle(buffer_size=10000).as_numpy_iterator().next()

In [ ]:
spectrogram, label = preprocess(filepath, label)

In [ ]:
plt.figure(figsize=(30,20))
plt.imshow(tf.transpose(spectrogram)[0])
plt.show()

Create a Tensorflow Data Pipeline

In [ ]:
data = data.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=100)
data = data.batch(16)
data = data.prefetch(8)

Split into Training and Testing Datasets

In [ ]:
train = data.take(18)
test = data.skip(18).take(7)

Load Tensorflow Dependencies

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

Build Sequential Model 

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation = 'relu', input_shape = (1491, 257, 1)))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), activation = 'relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

Compile Sequential Model

In [ ]:
model.compile('Adam', loss='BinaryCrossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

Summary Sequential Model

In [ ]:
model.summary()

Training Model

In [ ]:
hist = model.fit(train, epochs=4, validation_data=test)

Viewing and Plotting Graphs of Model Results

In [ ]:
plt.title('Loss')
plt.plot(hist.history['loss'], 'r')
plt.plot(hist.history['val_loss'], 'b')
plt.show()

In [ ]:
plt.title('Precision')
plt.plot(hist.history['precision'], 'r')
plt.plot(hist.history['val_precision'], 'b')
plt.show()

In [ ]:
plt.title('Recall')
plt.plot(hist.history['recall'], 'r')
plt.plot(hist.history['val_recall'], 'b')
plt.show()

Get one batch and make Predictions

In [ ]:
X_test, y_test = test.as_numpy_iterator().next()

In [ ]:
yhat = model.predict(X_test)

In [ ]:
yhat = [1 if prediction > 0.5 else 0 for prediction in yhat]


In [ ]:
yhat

Save Model

In [ ]:
from tensorflow.keras.models import load_model
model.save(os.path.join('model','ambulance_siren_classifier.h5'))

# Model Demo

Import Dependencies

In [ ]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
from itertools import groupby

Limit Cuda Memory Growth(Optional)

In [ ]:
phy = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(phy[0],True)

Loading Model

In [ ]:
os.chdir("D:\\AIML Projects\\Neural Networks\\Audio Classifier\\model")

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model('ambulance_siren_classifier.h5')

Load MP3 File

In [ ]:
def load_mp3_16k_mono(filename):
    res = tfio.audio.AudioIOTensor(filename)
    tensor = res.to_tensor()
    tensor = tf.math.reduce_sum(tensor, axis=1) / 2 
    sample_rate = res.rate
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(tensor, rate_in=sample_rate, rate_out=16000)
    return wav

Joining Paths for MP3 File

In [ ]:
mp3 = os.path.join('data', 'test', 'not_ambulance_new.mp3')

Resampling the MP3 File

In [ ]:
os.chdir("D:\\AIML Projects\\Neural Networks\\Audio Classifier")

In [ ]:
wav = load_mp3_16k_mono(mp3)

Slicing the MP3 File into smaller chunks

In [ ]:
audio_slices = tf.keras.utils.timeseries_dataset_from_array(wav, wav, sequence_length=48000, sequence_stride=48000, batch_size=1)

In [ ]:
samples, index = audio_slices.as_numpy_iterator().next()

Building the Preprocessing Function for MP3 File

In [ ]:
def preprocess_mp3(sample, index):
    sample = sample[0]
    zero_padding = tf.zeros([48000] - tf.shape(sample), dtype=tf.float32)
    wav = tf.concat([zero_padding, sample],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

Making Spectrogram for the MP3 File

In [ ]:
audio_slices = tf.keras.utils.timeseries_dataset_from_array(wav, wav, sequence_length=16000, sequence_stride=16000, batch_size=1)
audio_slices = audio_slices.map(preprocess_mp3)
audio_slices = audio_slices.batch(16)

Making Predictions on the MP3 File

In [ ]:
yhat = new_model.predict(audio_slices)
yhat = [1 if prediction > 0.95 else 0 for prediction in yhat]

In [ ]:
yhat

In [ ]:
yhat = [key for key, group in groupby(yhat)]
calls = tf.math.reduce_sum(yhat).numpy()

In [ ]:
if(calls>0):
    print("There is an Ambulance!!!")
else:
    print("There is no Ambulance.")

In [ ]:
results = {}
for file in os.listdir(os.path.join('data', 'test')):
    FILEPATH = os.path.join('data','test', file)
    
    wav = load_mp3_16k_mono(FILEPATH)
    audio_slices = tf.keras.utils.timeseries_dataset_from_array(wav, wav, sequence_length=48000, sequence_stride=48000, batch_size=1)
    audio_slices = audio_slices.map(preprocess_mp3)
    audio_slices = audio_slices.batch(16)
    
    yhat = new_model.predict(audio_slices)
    yhat = [1 if prediction > 0.9 else 0 for prediction in yhat]

    yhat = [key for key, group in groupby(yhat)]
    calls = tf.math.reduce_sum(yhat).numpy()

    if(calls>0):
        yhat = "There is an Ambulance!!!"
    else:
        yhat = "There is no Ambulance."
    
    results[file] = yhat

In [ ]:
results